<a href="https://colab.research.google.com/github/hvnpoet/colab/blob/master/tf_pose_estimation_with_webcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 웹캠, Colab, OpenPose 연동

웹캠으로 받은 영상을 Colabd과 연동하여 Colab에서 실행되는 OpenPose로 포즈를 추출하고 이를 노트북 화면으로 보여준다.

OpenPose 코드는 ildoonet님의 https://github.com/ildoonet/tf-pose-estimation를 사용하였고

webcam과 Colab 연동은 colab 가이드의 문서 https://colab.research.google.com/drive/1tbAeRge6KKgCYdC6ihDrsl80aRYoVOMa#scrollTo=QBFFBpx0TV5X&forceEdit=true&offline=true&sandboxMode=true를 참조하였다.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# OpenPose 설치

ildoonet님의 tf-pose-estimation을 다운로드 받고, 필요한 라이브러리를 설치한다.

In [2]:
%cd /content

import os
from os.path import exists, join, basename, splitext

if not exists('tf-pose-estimation'):
  !git clone https://github.com/ildoonet/tf-pose-estimation

%cd tf-pose-estimation
#!pip install -r requirements.txt
#!python setup.py install 
!pip install -e .

/content
Cloning into 'tf-pose-estimation'...
remote: Enumerating objects: 1362, done.
remote: Total 1362 (delta 0), reused 0 (delta 0), pack-reused 1362
Receiving objects: 100% (1362/1362), 94.13 MiB | 27.92 MiB/s, done.
Resolving deltas: 100% (729/729), done.
/content/tf-pose-estimation
Obtaining file:///content/tf-pose-estimation
     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 296kB 40.8MB/s 
  Created wheel for dill: filename=dill-0.2.7.1-cp36-none-any.whl size=54544 sha256=6d2a13d987aa2b7ca4d43d71eb0743130fc8995254731d61c3a2618d17ae55db
  Stored in directory: /root/.cache/pip/wheels/99/c4/ed/1b64d2d5809e60d5a3685530432f6159d6a9959739facb61f2
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=84b62a353a8f05eec0394abf6f66136cf3302b588aa38ede7bb0e11c8ad754e1
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2

# 모델 다운로드

4개의 모델이 사용 가능한데, 3개는 이미 포함되어 있고 나머지 'cmu' 1개만 따로 다운로드 받으면 된다.

그런데 그것 사용안한다면 다운로드 받지 않아도 된다.

In [0]:
# 4개의 모델이 가능하다.
# 이중 cmu 모델만 크기가 200M여서 다른곳에 있다.
# 다른 모델을 사용할 경우 다운로드 하지 않아도 된다.

# 파일 저장된 곳이 변경되었나 보다. 다운로드가 안된다.
#! cd models/graph/cmu && bash download.sh

# 다음 url에서 직접 다운받아서 파일을 colab에 올린다.
#      http://www.mediafire.com/file/qlzzr20mpocnpa3/graph_opt.pb

[download] model graph : cmu
--2019-04-15 07:15:07--  http://download2262.mediafire.com/m052rbtad9zg/qlzzr20mpocnpa3/graph_opt.pb
Resolving download2262.mediafire.com (download2262.mediafire.com)... 199.91.155.3
Connecting to download2262.mediafire.com (download2262.mediafire.com)|199.91.155.3|:80... connected.
HTTP request sent, awaiting response... 206 Partial Content
Retrying.

--2019-04-15 07:15:08--  (try: 2)  http://download2262.mediafire.com/m052rbtad9zg/qlzzr20mpocnpa3/graph_opt.pb
Connecting to download2262.mediafire.com (download2262.mediafire.com)|199.91.155.3|:80... connected.
HTTP request sent, awaiting response... 206 Partial Content
Giving up.

[download] end


# 수동 다운로드와 업로드

'cmu' 모델을 사용하지 않는다면 아래의 것도 실행하지 않아도 된다.


In [5]:
# 아래 코드로 노트북에서 직접 업로드할 수도 있으나, 엄청 오래 걸린다.
#from google.colab import files
#uploaded = files.upload()

# 구글 드라이브로 수동 업로드 하고 마운트해서 카피했다.
# 구글 드라이브 tmp에 업로드 하였다.
!ls -al "/content/drive/My Drive/data/tmp/graph_opt.pb"
!cp "/content/drive/My Drive/data/tmp/graph_opt.pb" /content/tf-pose-estimation/models/graph/cmu/
%cd ./tf-pose-estimation/

-rw------- 1 root root 209299198 Nov 27 10:58 '/content/drive/My Drive/data/tmp/graph_opt.pb'
/content/tf-pose-estimation


# OpenPose 동작 확인

실행시에 단지 오류 메시지만 뜨지 않으면 된다.

화면으로 보이는 것은 없다.

In [6]:
# 아래 코드는 실행 시 오류가 발생하지는 않는다.
# 다만 linux 콘솔에서 창을 띄워 결과를 보여준다.
!python run.py --model=mobilenet_thin --resize=432x368 --image=./images/p1.jpg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


[2019-11-27 14:12:36,695] [TfPoseEstimator] [INFO] loading graph from /content/tf-pose-estimation/models/graph/mobilenet_thin/graph_opt.pb(default size=432x368)
2019-11-27 14:12:36,695 INFO loading graph from /content/tf-pose-estimation/models/graph/mobilenet_thin/graph_opt.pb(default size=432x368)

2019-11-27 14:12:36,696 WARNING From /content/tf-pose-estimation/tf_pose/estimator.py:311: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


2019-11-27 14:12:36,696 WARNING From /content/tf-pose-estimation/tf_pose/estimator.py:312: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef inst

# OpenPose 패키지 방식의 사용법

아래 방식으로 사용할 수 있는데,
반환된 결과가  COCO 스타일이다.

TFEstimator.draw_human()과 같은 타 메소드를 사용 못한다.


In [8]:
#tensorflow_version default는 2.0이나 tf_pose에서는 1.x버전이 필요
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

import sys
sys.path.insert(0, "/content/tf-pose-estimation/tf_pose")
from tf_pose import common


image_path = './images/p1.jpg'
import tf_pose
humans = tf_pose.infer(image_path)
print(humans)


1.15.0
[([111, 38, 2, 113, 34, 2, 106, 34, 2, 119, 32, 2, 100, 32, 2, 129, 50, 2, 92, 52, 2, 145, 78, 2, 78, 68, 2, 145, 106, 2, 58, 74, 2, 135, 106, 2, 108, 108, 2, 145, 144, 2, 102, 144, 2, 155, 180, 2, 115, 192, 2], 1.7460929155349731)]


# OpenPose 호출 함수 선언

웹캠으로 전달받은 이미지를 OpenPose로 처리하기 위한 함수를 선언한다.

In [9]:
import sys
sys.path.insert(0, "/content/tf-pose-estimation/tf_pose")

from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path

w = 400
h = 300

e = TfPoseEstimator(get_graph_path('mobilenet_v2_large'), target_size=(w, h))



def extract_pose(image):
  humans = e.inference(image, resize_to_default=(w>0 and h>0), upsample_size=4.0)
  return humans
  
def get_pose_extracted_image(image):
  humans = extract_pose(image)
  pose_extracted_image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
  %matplotlib inline 
  from matplotlib.pyplot import imshow
  imshow(pose_extracted_image)

  return pose_extracted_image
  

[2019-11-27 14:30:20,511] [TfPoseEstimator] [INFO] loading graph from /content/tf-pose-estimation/models/graph/mobilenet_v2_large/graph_opt.pb(default size=400x300)
2019-11-27 14:30:20,511 INFO loading graph from /content/tf-pose-estimation/models/graph/mobilenet_v2_large/graph_opt.pb(default size=400x300)


TfPoseEstimator/Openpose/concat_stage7/axis
TfPoseEstimator/Mconv7_stage6_L2/biases
TfPoseEstimator/Mconv7_stage6_L2/weights
TfPoseEstimator/Mconv6_stage6_L2/biases
TfPoseEstimator/Mconv6_stage6_L2/weights
TfPoseEstimator/Mconv5_stage6_L2/biases
TfPoseEstimator/Mconv5_stage6_L2/weights
TfPoseEstimator/Mconv4_stage6_L2/biases
TfPoseEstimator/Mconv4_stage6_L2/weights
TfPoseEstimator/Mconv3_stage6_L2/biases
TfPoseEstimator/Mconv3_stage6_L2/weights
TfPoseEstimator/Mconv2_stage6_L2/biases
TfPoseEstimator/Mconv2_stage6_L2/weights
TfPoseEstimator/Mconv1_stage6_L2/biases
TfPoseEstimator/Mconv1_stage6_L2/weights
TfPoseEstimator/Mconv7_stage6_L1/biases
TfPoseEstimator/Mconv7_stage6_L1/weights
TfPoseEstimator/Mconv6_stage6_L1/biases
TfPoseEstimator/Mconv6_stage6_L1/weights
TfPoseEstimator/Mconv5_stage6_L1/biases
TfPoseEstimator/Mconv5_stage6_L1/weights
TfPoseEstimator/Mconv4_stage6_L1/biases
TfPoseEstimator/Mconv4_stage6_L1/weights
TfPoseEstimator/Mconv3_stage6_L1/biases
TfPoseEstimator/Mconv3_st

# 웹화면 구성

2개의 엘리멘트를 생성한다.

웹캠으로 받은 영상을 그대로 보여주는  'video'와 OpenPose로 처리한 것을 보여주는'output'이름의  'img'
  
  

In [0]:
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode

VIDEO_HTML = """
<video autoplay
 width=400 height=300></video>
<br>
<img id="output" src="https://ko.wikipedia.org/static/images/wikimedia-button.png"/>


<script>
var video = document.querySelector('video')
navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)

var canvas = document.createElement('canvas')
var [w,h] = [video.offsetWidth, video.offsetHeight]
canvas.width = w
canvas.height = h


</script>
"""

DATA_HTML = """
<script>
canvas.getContext('2d').drawImage(video, 0, 0, w, h)
var data = canvas.toDataURL('image/jpeg', %f)
</script>
"""

# 웹캠 포즈 추츨 실행

위에서 설정한 video에서 데이터를 읽고, OpenPose를 호출하여 처리한 이미지를 표시한다.

In [0]:
from IPython.display import Image, display
from matplotlib import pyplot as plt
import cv2
import numpy as np
import base64

%matplotlib inline

def convert_jpeg_binary_to_cv2_image(binary_image):
  return cv2.imdecode(np.fromstring(binary_image, dtype=np.uint8), -1)


def show_pose_extracted(quality=1.0):
  display(HTML(VIDEO_HTML))
  while True:
    display(HTML(DATA_HTML % quality))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    cv2_image = convert_jpeg_binary_to_cv2_image(binary)
    
    extracted = get_pose_extracted_image(cv2_image)
    
    binanry_img = cv2.imencode('.jpg', extracted)[1]
    
    b64 = base64.b64encode(binanry_img).decode('ascii')
    display(HTML('''<script>document.getElementById("output").src="data:image/jpg;base64, {}"</script>'''.format(b64)))
    if data is None: break

  
show_pose_extracted()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  # Remove the CWD from sys.path while we load stuff.
